In [ ]:
import pandas as pd
import plotly.express as px
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook+pdf"

In [ ]:
reg_config = r'VOL ([0-9]+)'
reg_size1_patterns = r'Number of Patterns : ([0-9]+)'
reg_extented_sequence = r'\(([0-9]+) elements\)'
reg_exec_time_size_1 = r'Execution time size 1 pattern: ([0-9]+)s'
reg_exec_time_seq = r'Execution time : ([0-9]+)s'



In [ ]:
lines = []
with open("../analyzed_results/volume_impact") as file:
    current_line = ""
    for line in file :
        
        if len(line)>1: 
            if re.match(reg_config, line):
                if len(current_line)>1:
                    lines.append(current_line)
                current_line=line + " "
            else :
                current_line+=line.strip()+" "
lines.append(current_line)
                

In [ ]:
def getInformations(line):
    match_config = re.findall(reg_config, line)
    nbTS = match_config[0]
    # nb_att = int(match_config[0][1])
    # vol = int(match_config[0][2])
    time_1 = int(re.findall(reg_exec_time_size_1, line)[0])
    time_seq = 0
    current_threshold = int(match_config[0])
    if 'OOME' in line:
        time_seq = -1
    else :
        time_seq = int(re.findall(reg_exec_time_seq, line)[0])
    match_sequences = re.findall(reg_extented_sequence, line)
    sum = 0
    for m in match_sequences:
        sum += int(m) if int(m) < current_threshold else current_threshold
    # nb_subgraph = int(re.findall(reg_size1_patterns, line)[0])

    dic = {}
    # dic['Dataset'] = dataset
    dic['Volume'] = current_threshold
    # dic['Support'] = vol
    dic['Timestamps'] = 10
    dic['Time Extraction Subgraphs'] = time_1
    dic['Time Sequence'] = time_seq
    dic['Nb Sequence'] = sum
    # dic['Nb Subgraph'] = nb_subgraph
    return dic
    


In [ ]:
li_of_dict = []
for i, l in enumerate(lines) :
    print(i)
    li_of_dict.append(getInformations(l))

In [ ]:
df = pd.DataFrame(li_of_dict)
df["Total Time"] = df["Time Extraction Subgraphs"] + df["Time Sequence"]
df


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# df.sort_values(by='Support', inplace=True)
    # print(tmp_df)
    # Add traces
fig.add_trace(
    go.Scatter(x=df["Volume"], y=df["Time Sequence"], 
               name="Execution Time(s)", 
               mode="lines+markers+text", 
               textposition="top center",
               text =df["Time Sequence"],
               line=dict(width=8)),
    secondary_y=False,
)

fig.update_layout(
width=1486, height=525,
    # title_text="Compared Execution Time for Subgraph Exctraction for Covid-19 dataset", 
        font=dict(
        # family="Courier New, monospace",
        size=30,  # Set the font size here
        # color="RebeccaPurple"
    ), 
        yaxis_range=[0,25000], 
        legend=dict(
        x=0,
        y=1,
        traceorder="reversed",
        # title_font_family="Times New Roman",
        # font=dict(
        #     family="Courier",
        #     size=12,
        #     color="black"
        # ),
        # bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=1
    )
    
)
# fig.add_trace(
#     go.Scatter(x=df["Volume"], y=df["Nb Sequence"], name="Number of sequences extentded"),
#     secondary_y=True,
# )

    # Add figure title
# fig.update_layout(
#         title_text=f"Dataset : {dataset} Impact of Support on computation time and sequence results"
# )

    # Set x-axis title
fig.update_xaxes(title_text="Volume", )
fig.update_yaxes(title_text="Execution Time (s)", secondary_y=False, )
fig.update_yaxes(title_text="Number of sequences extentded", secondary_y=True,)
fig.show()

In [ ]:
df